In [1]:
from quantrocket import get_prices
import pandas as pd
import numpy as np

class MarketModel:
    def __init__(self, ticker, start_date, end_date):
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
        self.close_prices = None
        self.returns = None
        self.states = None
        self.state_transitions = None
        self.portfolio_value = 0
        self.optimal_buy_indices = []

    def fetch_data(self):
        apple_data = get_prices("usstock-free-1d", universes="usstock-free", start_date=self.start_date, end_date=self.end_date, fields=["Close"])
        self.close_prices = apple_data[self.ticker]

    def calculate_returns(self):
        self.returns = self.close_prices.pct_change()

    def classify_states(self):
        self.states = np.where(self.returns >= 0.01, 1, np.where(self.returns > -0.01, 0, -1))

    def implement_value_function(self):
        self.state_transitions = np.zeros((3, 3))
        for i in range(1, len(self.states)):
            if self.states[i] == 1 and self.states[i-1] == 0:
                self.portfolio_value += 1
                self.optimal_buy_indices.append(i)
            elif self.states[i] == -1 and self.states[i-1] == 0:
                self.portfolio_value -= 1
            self.state_transitions[self.states[i-1] + 1, self.states[i] + 1] += 1

    def calculate_transition_distribution(self):
        self.transition_distribution = np.zeros_like(self.state_transitions)
        for i in range(len(self.state_transitions)):
            row_sum = np.sum(self.state_transitions[i])
            if row_sum != 0:
                self.transition_distribution[i] = self.state_transitions[i] / row_sum


    def print_results(self):
        print("Portfolio Value:", self.portfolio_value)
        print("Optimal Index:", self.optimal_buy_indices)
        print("Transition Distribution:")
        print(pd.DataFrame(self.transition_distribution, index=['Bear', 'Flat', 'Bull'], columns=['Bear', 'Flat', 'Bull']))
        
if __name__ == "__main__":
    model = MarketModel('FIBBG000B9XRY4', '2023-01-01', '2023-12-31')
    model.fetch_data()
    model.calculate_returns()
    model.classify_states()
    model.implement_value_function()
    model.calculate_transition_distribution()
    model.print_results()


Portfolio Value: 0
Optimal Index: []
Transition Distribution:
      Bear  Flat  Bull
Bear   0.0   1.0   0.0
Flat   0.0   1.0   0.0
Bull   0.0   0.0   0.0


In [2]:
    model = MarketModel('FIBBG000B9XRY4', '2020-01-01', '2020-12-31')
    model.fetch_data()
    model.calculate_returns()
    model.classify_states()
    model.implement_value_function()
    model.calculate_transition_distribution()
    model.print_results()

Portfolio Value: 3
Optimal Index: [49, 56, 146]
Transition Distribution:
          Bear      Flat      Bull
Bear  0.000000  1.000000  0.000000
Flat  0.000000  0.987854  0.012146
Bull  0.333333  0.666667  0.000000


In [3]:
    model = MarketModel('FIBBG000B9XRY4', '2019-01-01', '2019-12-31')
    model.fetch_data()
    model.calculate_returns()
    model.classify_states()
    model.implement_value_function()
    model.calculate_transition_distribution()
    model.print_results()

Portfolio Value: 0
Optimal Index: []
Transition Distribution:
      Bear  Flat  Bull
Bear   0.0   1.0   0.0
Flat   0.0   1.0   0.0
Bull   0.0   0.0   0.0


In [4]:
    model = MarketModel('FIBBG000B9XRY4', '2007-01-01', '2023-12-31')
    model.fetch_data()
    model.calculate_returns()
    model.classify_states()
    model.implement_value_function()
    model.calculate_transition_distribution()
    model.print_results()

Portfolio Value: 3
Optimal Index: [218, 448, 478, 3321, 3328, 3418]
Transition Distribution:
          Bear      Flat      Bull
Bear  0.000000  1.000000  0.000000
Flat  0.000703  0.997890  0.001406
Bull  0.166667  0.833333  0.000000
